https://github.com/nomomon/Amazon-Product-Description/tree/main

In [ ]:
!git clone https://github.com/nomomon/Amazon-Product-Description.git

In [ ]:
%cd /kaggle/working/Amazon-Product-Description

https://jmcauley.ucsd.edu/data/amazon_v2/index.html

In [ ]:
import urllib.request

def download_file(category): 
    metadata_url = "https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles/meta_"+ category +".json.gz" 
    urllib.request.urlretrieve(metadata_url, './data.json.gz')

In [ ]:
import gzip 
import json

def read_file():
    counter = 0
    data = []
    with gzip.open('data.json.gz') as f:
        for l in f:
            if counter < 5000:
                data.append(json.loads(l.strip()))
                counter+=1
        
    return data

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
def clean(item):
    if type(item["description"]).__name__ == "str":
        item["description"] = [item["description"]]
    if type(item["category"]).__name__ == "str":
        item["category"] = [item["category"]]
    if type(item["feature"]).__name__ == "str":
        item["feature"] = [item["feature"]]
    return item

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
def convert_to_prompt(item):
    title = item["title"]
    categories = ", ".join(item["category"])
    feature = ", ".join(item["feature"])
    description = ". ".join(item["description"])

    description = cleanhtml(description)
    categories = cleanhtml(categories)
    feature = cleanhtml(feature)


    if len(categories) > 10 and len(title) > 10 and len(description.split(' ')) >= 10 and len(description.split(' ')) <= 60 and len(feature) > 10:
        prompt = f'''Categories: {categories}
Title: {title}
Description: {description}
Features: {feature}
<|endoftext|>'''
    else:
        prompt = ""

    return prompt

Put to file

In [ ]:
def put_to_file(category, text):
    f = open(f"dataset/{category}.txt", "a+")
    f.write(text)
    f.close()

Merge everything

In [ ]:
categories = [
    "Electronics",
    "Home and Kitchen",
    "Toys and Games",
]

done = []

In [ ]:
import os, shutil
folder = 'dataset'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
def doAll():
    for category in categories:
        if category not in done:    
            print(f"Doing {category} ...")

            subCat = category.replace(" ", "_")
            download_file(subCat)
            print("     Downloaded")
            for item in read_file():
                put_to_file(subCat, convert_to_prompt(clean(item)))
            print("     Placed in a file")

            done.append(category)

doAll()

Split Dataset Train dan Dataset Test

In [ ]:
%mkdir dataset_train

In [ ]:
%mkdir dataset_test

In [ ]:
def parse_dataset(file_path, separator):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Gunakan string pemisah untuk membagi konten file
        entries = file.read().strip().split(separator)

    parsed_data = []
    for entry in entries:
        lines = entry.split('\n')
        entry_dict = {}
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                entry_dict[key] = value
        parsed_data.append(entry_dict)

    return parsed_data

def split_dataset(dataset, first_split_size):
    return dataset[:first_split_size], dataset[first_split_size:500]

def write_dataset_to_file(dataset, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in dataset:
            for key, value in entry.items():
                file.write(f"{key}: {value}\n")
            file.write("<|endoftext|>\n")  # Menambahkan pemisah antar entri
            
for category in categories:
    subCat = category.replace(" ", "_")
    file_path = f'dataset/{subCat}.txt'
    separator = '<|endoftext|>'
    dataset = parse_dataset(file_path, separator)
    dataset_200, dataset_100 = split_dataset(dataset, 400)
    write_dataset_to_file(dataset_200, f'dataset_train/{subCat}.txt')
    write_dataset_to_file(dataset_100, f'dataset_test/{subCat}.txt')

Merge dataset with copy *.txt Merge{Train/Test}Dataset.txt

GPT NEO

In [2]:
pip install happytransformer

  Obtaining dependency information for happytransformer from https://files.pythonhosted.org/packages/f6/ed/8abe77d280294a454534003242431439b102cf17e1089fde6020f90ab621/happytransformer-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for datasets<3.0.0,>=2.13.1 from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from happytransformer import HappyGeneration

In [ ]:
happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")

In [ ]:
from happytransformer import GENTrainArgs

In [ ]:
train_args = GENTrainArgs(learning_rate = 3e-6, num_train_epochs = 1)
happy_gen.train("/kaggle/input/merge-train-dataset-6-0/MergeTrainDataset.txt", args = train_args)

In [ ]:
happy_gen.save("/kaggle/working/GPT-NEO_Amazon/")

Test Model

In [3]:
!pip install Rouge

In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from happytransformer import HappyGeneration

happy_gen = HappyGeneration(load_path="/kaggle/input/gpt-neo-model-skripsi-6-0")

In [ ]:
from torchinfo import summary
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('EleutherAI/gpt-neo-125M')
summary(model)

In [5]:
from happytransformer import GENSettings

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from rouge import Rouge

def parse_dataset(file_path, separator):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Gunakan string pemisah untuk membagi konten file
        entries = file.read().strip().split(separator)

    parsed_data = []
    for entry in entries:
        lines = entry.split('\n')
        entry_dict = {}
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                entry_dict[key] = value
        parsed_data.append(entry_dict)

    return parsed_data

dataset = parse_dataset('/kaggle/input/merge-test-dataset-6-0/MergeTestDataset.txt', '<|endoftext|>')

def calculate_rouge_l_f_scores(model_summaries, reference_summaries):
    rouge = Rouge()
    scores = rouge.get_scores(model_summaries, reference_summaries)
    return [score['rouge-l']['f'] for score in scores]

def calculate_bleu_scores(model_summaries, reference_summaries):
    bleu_scores = []
    for model, reference in zip(model_summaries, reference_summaries):
        # Tokenize the sentences
        model_tokens = nltk.word_tokenize(model)
        reference_tokens = [nltk.word_tokenize(reference)]
        
        # Calculate BLEU score
        score = sentence_bleu(reference_tokens, model_tokens)
        bleu_scores.append(score)
    
    return bleu_scores

def test_model():
    model_summaries = []
    reference_summaries = []
    model_summaries_2 = []
    reference_summaries_2 = []
    model_summaries_3 = []
    reference_summaries_3 = []

    for data in dataset:
        if data == {}: 
            continue
        if len(data['Description'].split(' ')) < 10:
            continue
    
        descs = data['Description'].split(' ')
    
        text = f"""Categories: {data['Categories']}
        Title: {data['Title']}
        Features: {data['Features']}
        Description: """
    
        result = happy_gen.generate_text(text, args=gen_args)
        output = result.text

        if data['Categories'].split(',')[0] == "Electronics":
            model_summaries.append(output) # Replace this with your model-generated summaries
            reference_summaries.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Home & Kitchen":
            model_summaries_2.append(output) # Replace this with your model-generated summaries
            reference_summaries_2.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Toys & Games":
            model_summaries_3.append(output) # Replace this with your model-generated summaries
            reference_summaries_3.append(data['Description']) # Replace this with your reference summaries
    
    f_scores = calculate_rouge_l_f_scores(model_summaries, reference_summaries)
    f_scores_2 = calculate_rouge_l_f_scores(model_summaries_2, reference_summaries_2)
    f_scores_3 = calculate_rouge_l_f_scores(model_summaries_3, reference_summaries_3)

    bleu_scores = calculate_bleu_scores(model_summaries, reference_summaries)
    bleu_scores_2 = calculate_bleu_scores(model_summaries_2, reference_summaries_2)
    bleu_scores_3 = calculate_bleu_scores(model_summaries_3, reference_summaries_3)
    
    return f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3

In [8]:
from rouge import Rouge

def parse_dataset(file_path, separator):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Gunakan string pemisah untuk membagi konten file
        entries = file.read().strip().split(separator)

    parsed_data = []
    for entry in entries:
        lines = entry.split('\n')
        entry_dict = {}
        for line in lines:
            if ': ' in line:
                key, value = line.split(': ', 1)
                entry_dict[key] = value
        parsed_data.append(entry_dict)

    return parsed_data

dataset = parse_dataset('/kaggle/input/merge-test-dataset-6-0/MergeTestDataset.txt', '<|endoftext|>')

def calculate_rouge_l_f_scores(model_summaries, reference_summaries):
    rouge = Rouge()
    scores = rouge.get_scores(model_summaries, reference_summaries)
    return [score['rouge-l']['f'] for score in scores]

def calculate_bleu_scores(model_summaries, reference_summaries):
    bleu_scores = []
    for model, reference in zip(model_summaries, reference_summaries):
        # Tokenize the sentences
        model_tokens = nltk.word_tokenize(model)
        reference_tokens = [nltk.word_tokenize(reference)]
        
        # Calculate BLEU score
        score = sentence_bleu(reference_tokens, model_tokens)
        bleu_scores.append(score)
    
    return bleu_scores

def test_model():
    model_summaries = []
    reference_summaries = []
    model_summaries_2 = []
    reference_summaries_2 = []
    model_summaries_3 = []
    reference_summaries_3 = []

    for data in dataset:
        if data == {}: 
            continue
        if len(data['Description'].split(' ')) < 10:
            continue
    
        descs = data['Description'].split(' ')
    
        text = f"""Categories: {data['Categories']}
        Title: {data['Title']}
        Features: {data['Features']}
        Description: {descs[0]} {descs[1]}"""
    
        result = happy_gen.generate_text(text, args=gen_args)
        output = result.text

        if data['Categories'].split(',')[0] == "Electronics":
            model_summaries.append(output) # Replace this with your model-generated summaries
            reference_summaries.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Home & Kitchen":
            model_summaries_2.append(output) # Replace this with your model-generated summaries
            reference_summaries_2.append(data['Description']) # Replace this with your reference summaries
        if data['Categories'].split(',')[0] == "Toys & Games":
            model_summaries_3.append(output) # Replace this with your model-generated summaries
            reference_summaries_3.append(data['Description']) # Replace this with your reference summaries
    
    f_scores = calculate_rouge_l_f_scores(model_summaries, reference_summaries)
    f_scores_2 = calculate_rouge_l_f_scores(model_summaries_2, reference_summaries_2)
    f_scores_3 = calculate_rouge_l_f_scores(model_summaries_3, reference_summaries_3)

    bleu_scores = calculate_bleu_scores(model_summaries, reference_summaries)
    bleu_scores_2 = calculate_bleu_scores(model_summaries_2, reference_summaries_2)
    bleu_scores_3 = calculate_bleu_scores(model_summaries_3, reference_summaries_3)
    
    return f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3

In [9]:
def evaluation_metric():
    rouge_avg = []
    rouge_avg.append(np.mean(f_scores))
    rouge_avg.append(np.mean(f_scores_2))
    rouge_avg.append(np.mean(f_scores_3))

    bleu_avg = []
    bleu_avg.append(np.mean(bleu_scores))
    bleu_avg.append(np.mean(bleu_scores_2))
    bleu_avg.append(np.mean(bleu_scores_3))

    evaluation_score = {
        "Rouge Average": rouge_avg,
        "Bleu Average": bleu_avg
    }

    df = pd.DataFrame(evaluation_score, index = ["Electronics","Home and Kitchen","Toys and Games",])
    return df

In [10]:
def std():
    rouge_std = []
    rouge_std.append(np.std(f_scores))
    rouge_std.append(np.std(f_scores_2))
    rouge_std.append(np.std(f_scores_3))

    bleu_std = []
    bleu_std.append(np.std(bleu_scores))
    bleu_std.append(np.std(bleu_scores_2))
    bleu_std.append(np.std(bleu_scores_3))

    evaluation_score = {
        "Rouge Standard Deviation": rouge_std,
        "Bleu Standard Deviation": bleu_std
    }

    df = pd.DataFrame(evaluation_score, index = ["Electronics","Home and Kitchen","Toys and Games",])
    return df

Experiment Temperature

In [11]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 1.4
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [12]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_t

In [13]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.099872,0.498787
Home and Kitchen,0.084050,0.455414
Toys and Games,0.115376,0.425324


In [14]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.041736,0.158243
Home and Kitchen,0.044417,0.151742
Toys and Games,0.049195,0.188135


In [15]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [16]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [17]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.117183,0.473953
Home and Kitchen,0.099270,0.421236
Toys and Games,0.120552,0.343014


In [18]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.047593,0.180452
Home and Kitchen,0.062299,0.194247
Toys and Games,0.060102,0.177183


In [19]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.1
top_k = 50
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [20]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [21]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.119554,0.458407
Home and Kitchen,0.102493,0.383851
Toys and Games,0.133255,0.379161


In [ ]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.059111,0.204648
Home and Kitchen,0.064974,0.174121
Toys and Games,0.063338,0.177912


Experiment Top_K

In [ ]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 1
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [ ]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [ ]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.125202,0.448928
Home and Kitchen,0.105767,0.420960
Toys and Games,0.132710,0.369187


In [ ]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.054485,0.200735
Home and Kitchen,0.060397,0.171006
Toys and Games,0.061497,0.179608


In [ ]:
min_length =  50
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.7
top_k = 100
top_p = 0.7
no_repeat_ngram_size = 2

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [ ]:
f_scores, f_scores_2, f_scores_3, bleu_scores, bleu_scores_2, bleu_scores_3 = test_model()

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 f

In [ ]:
df = evaluation_metric()
df

,Rouge Average,Bleu Average
Electronics,0.105694,0.489093
Home and Kitchen,0.097540,0.380725
Toys and Games,0.124578,0.360013


In [ ]:
df = std()
df

,Rouge Standard Deviation,Bleu Standard Deviation
Electronics,0.057662,0.181627
Home and Kitchen,0.063138,0.176161
Toys and Games,0.055451,0.194041
